In [ ]:
import pandas as pd
import os
import sys
import zipfile
import json
from tqdm.auto import tqdm
tqdm.pandas()
from ipywidgets import interact
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

Read the table with P numbers of tablets that belong to the Treasure Archive and Shoe Archive. Retrieve a list of numbers.

In [ ]:
file = 'csv/treasury_shoes2.txt'
tr_df = pd.read_csv(file, encoding='utf8')
ids = list(tr_df['id_text'])
ids.sort()

If necessary, download the Ur3 JSON from epsd2. If the file is already in the directory `/jsonzip`, skip this step, but do assign 'epsd2/admin/ur3' to the variable `project`.

In [ ]:
project = 'epsd2/admin/ur3'
#oracc_download([project]);

Function for parsing the JSON. Words that appear in year names are marked in the field `ftype`.

In [ ]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:
                lemma['ftype'] = JSONobject['ftype'] # this picks up YN for year name
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

Call the parser for each of the P numbers in the list `ids`. The parser function needs the lists `lemm_l` and `dollar_keys`, as well as the dictionary `meta_d`. These objects are manipulated during the parsing process.

In [ ]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
file = f"jsonzip/{project.replace('/', '-')}.zip"
try:
    z = zipfile.ZipFile(file) 
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist() # list of all the files in the ZIP file
files = [name for name in files if name[-12:-5] in ids] # select only those of the treasury/leather archive
for filename in tqdm(files, desc = project):
    id_text = project + filename[-13:-5] 
    meta_d["id_text"] = id_text
    try:
        st = z.read(filename).decode('utf-8')
        data_json = json.loads(st)           
        parsejson(data_json)
    except:
        print(f'{id_text} is not available or not complete')
z.close()

The function `parsejson()` fills the lists `lemm_l` with data. Read this list into a DataFrame.

In [ ]:
words = pd.DataFrame(lemm_l).fillna('')
keep = ['extent', 'scope', 'state', 'id_word', 'id_text', 'form', 'cf', 'gw', 'pos', 'ftype', 'label']
words = words[keep]

Remove comma's and spaces from Guide words

In [ ]:
words['gw'] = words['gw'].replace([' ', ','], ['', ''], regex=True)

Simplify `id_text` ('P123456' instead of 'epsd2/admin/ur3/P123456') and add a field `id_line`.

In [ ]:
words['id_text'] = [i[-7:] for i in words['id_text']]
words['id_line'] = [int(i.split('.')[1]) for i in words['id_word']]

In [ ]:
proper_nouns = ['RN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'CN', 'GN', 'SN']
physical_break = ['illegible', 'traces', 'missing', 'effaced']
logical_break = ['other', 'blank', 'ruling']
words['lemma'] = words["cf"] + '[' + words["gw"] + ']' + words["pos"]
words.loc[words["cf"] == "" , 'lemma'] = words['form'] + '[NA]NA'
words.loc[words["pos"] == "n" , 'lemma'] = words['form'] + '[]NU'
words.loc[words["state"].isin(logical_break), 'lemma'] = "break_logical"
words.loc[words["state"].isin(physical_break), 'lemma'] = "break_physical"
words.head(10)

Read list of name forms and Normalized names

In [ ]:
normdf = pd.read_csv('Normalized/drehem_norm_names.csv', encoding='utf8')
normdf

Download OGSL

In [ ]:
oracc_download(['ogsl']);

List Ur 3 sign equivalencies

In [ ]:
equiv = {'ANŠE' : 'GIR₃', 
        'DUR₂' : 'KU', 
        'NAM₂' : 'TUG₂', 
        'TIL' : 'BAD', 
        'NI₂' : 'IM',
        'ŠAR₂' : 'HI', 
        }
w = re.compile(r'\w+') # replace whole words only - do not replace TILLA with BADLA.
           # but do replace |SAL.ANŠE| with |SAL.GIR₃|

Parse OGSL

In [ ]:
def parseogsljson(data_json):
    for key, value in data_json["signs"].items():
        key = re.sub(w, lambda m: equiv.get(m.group(), m.group()), key)
        if "values" in value:
            for n in value["values"]:
                d2[n] = key
    return

Create OGSL dictionary key = sign value, value = sign name.

In [ ]:
d2 = {}  # this empty dictionary is filled by the parsejson() function, called in this cell.
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parseogsljson(data_json)  
with open('output/ogsl_dict.p', 'wb') as p:
    pickle.dump(d2, p)  

In [ ]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']  # used in compound signs
#operators = ['&', '%', '@', '×']
flags = "][?<>⸢⸣⌈⌉*/" # note that ! is omitted from flags, because it is dealt with separately
table = str.maketrans(dict.fromkeys(flags))

In [ ]:
def signnames(translit):  
    """This function takes a string of transliterated cuneiform text and translates that string into a string of
    sign names, separated by spaces. In order to work it needs the variables separators, separators2, and table defined above. The variable table
    is used by the translate() method to translate all flags (except for !) to None. The function also needs a dictionary, called d2, that has as
    keys sign readings and sign names as corresponding values. In case a key is not found, the sign reading is replaced by itself."""
    signnames_l = []
    translit = translit.translate(table).lower()  # remove flags, half brackets, square brackets.
    translit = translit.replace('...', 'x')
    for s in separators: # split transliteration line into signs   
        translit = translit.replace(s, ' ').strip()
    s_l = translit.split() # s_l is a list that contains the sequence of transliterated signs without separators or flags
    s_l = [d2.get(sign, sign) for sign in s_l] # replace each transliterated sign with its sign name.
    # Now take care of some special situations: signs with qualifiers, compound signs.
    for sign in s_l:
        if '!' in sign: # corrected sign, as in ka!(SAG), get only the corrected reading.
            sign = sign.split('!(')[0]
            sign = sign.replace('!', '') # remove remaining exclamation marks
        elif sign[-1] == ')' and '(' in sign: # qualified sign, as in ziₓ(SIG₇) - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '×' in sign: #compound. Compound like |KA×NINDA| to be replaced by |KA×GAR|
            sign_l = sign.replace('|', '').split('×')
            #replace individual signs of the compound by OGSL names
            sign_l = [d2.get(sign, sign) for sign in sign_l] 
            # if user enters |KA*EŠ| this is transformed to ['KA', '|U.U.U|']. The pipes around U.U.U must be replaced by brackets
            sign_l = [f'({sign[1:-1]})' if len(sign) > 1 and sign[0] == '|' else sign for sign in sign_l]
            sign = f"|{'×'.join(sign_l)}|"  #put the sign together again with enclosing pipes.
        elif '.' in sign or '+' in sign: # using elif, so that compounds like |UD×(U.U.U)| are not further analyzed.
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()  # compound sign split into multiple signs
            sign_l = [d2.get(sign, sign) for sign in sign_l]
            for se in separators2:   # in case d2.get returns a compound sign name
                sign_l = [si.replace(se, ' ').strip() for si in sign_l]
            signnames_l.extend(sign_l)
            continue
        sign = d2.get(sign, sign)
        signnames_l.append(sign)
    # add space before and after each line so that each sign representation is enclosed in spaces
    signnames = f" {' '.join(signnames_l).upper()} " 
    return signnames

Make a new field in the Normalized Names table, representing the sign sequence (sign names) of the transliteration of each name.

In [ ]:
normdf["sign_names"] = normdf["transliteration"].progress_map(signnames)

Create a dictionary with `sign_names` as key and `normalization` as value.

In [ ]:
normd2 = dict(zip(normdf['sign_names'], normdf['normalization']))

Add a column to `words` with the sequence of sign names for each word.

In [ ]:
words['sign_names'] = words['form'].progress_map(signnames)

Use the `normd2` dictionary to transform sign name sequences into normalized names.

In [ ]:
#words.loc[(words.pos.isin(proper_nouns + ['X'])) & (words.lemma.str.contains('.')), 
#          'lemma'] = words.progress_apply(lambda x: normd2.get(x['sign_names'], x['lemma']), axis=1)
words.loc[words.pos.isin(proper_nouns + ['X']), 'lemma'] = words.progress_apply(lambda x: normd2.get(x['sign_names'], x['lemma']), axis=1)

Use the (temporary) corrections.csv file to apply corrections to the lemmatization

In [ ]:
corrections = pd.read_csv('Normalized/corrections.csv', encoding='utf8')
corr_d = dict(zip(corrections['form'], corrections['corr']))
words.loc[words.pos.isin(['PN', 'RN', 'X', 'DN']), 'lemma'] = words.progress_apply(lambda x: corr_d.get(x['form'], x['lemma']), axis =1)
words['pos'] = [w.split(']')[-1] if ']' in w else '' for w in words['lemma']]

In [ ]:
texttype = []
for i in ids:
    ttype = ''
    text = list(words.loc[words.id_text == i, 'lemma'])
    if 'mu.DU[delivery]N' in text:
        ttype = 'intake'
    elif 'zig[rise]V/i' in text:
        ttype = 'expenditure'
    elif 'teŋ[near]V/i' in text: 
        ttype = 'transfer'
    texttype.append(ttype)
treasure = dict(zip(ids, texttype))

Select PNs (and RNs) but skip those that appear in Year Names or Seals. Add lugal (king) sukkalmah (prime minister) and nin (queen).

In [ ]:
unnamed = {'lugal[king]N', 'nin[queen]N', 'sukkalmah[official]N'}
names = set(words.loc[words.pos.isin(['PN', 'RN', 'DN']), 'lemma'])
actors = unnamed | names
seal = list(words.loc[words.label.str.contains('seal'), 'id_word'])
yn = list(words.loc[words.ftype == 'yn', 'id_word'])
exclude = seal + yn
PNs = words.loc[(words.lemma.isin(actors)) & (~words.id_word.isin(exclude))].copy()

# Define keywords and the corresponding roles

In [ ]:
key_post = {'mu.DU[delivery]N': 'deliverer', 'maškim[administrator]N' : 'representative', 
               'zig[rise]V/i' : 'expender',
               'šu[hand]N teŋ[near]V/i': 'recipient' , 'šu[hand]N us[follow]V/t': 'sender'}
key_pre =  {'ŋiri[foot]N' : 'intermediary', 
            'arua[offering]N' : 'offerer', 'kišib[seal]N' : 'sealer',
            'mu[name]N' : 'reason', 'ki[place]N' : 'source'}

# Determine the Role of each PN
TODO: default role in texttype transfer. Check default roles for other text types.

In [ ]:
role = []
attribute = []
for i in PNs.index:                           # the index of PNs is identical to the one of words
    Pno = PNs.loc[i]['id_text']                    # the text ID (P number)
    lineno = PNs.loc[i]['id_line']                 # the line number inwhich the PN appears
    position_in_line = int(PNs.loc[i]['id_word'].split('.')[-1]) # the position of the PN in its line (starting at 1)
    text = words.loc[words.id_text == Pno]                # the entire text
    line = list(text.loc[text.id_line == lineno, 'lemma']) # list of words that appear in the line with the PN
    line_i = text.loc[text.id_line == lineno].index  # indexes of words in line
    mx = text.index[-1]                    # highest index no. of word in text
    mxl = text.loc[mx]['id_line']          # highest line number in text
    if lineno < mxl:                       # check that nextline is still in the same text
        nextl = words.loc[line_i[-1]+1]['id_line']
        nextline = list(text.loc[text.id_line == nextl, 'lemma']) # list of words in nextline
    if not nextline:
        nextline = ['none']                # nextline needs content
    attr = []
    r = ''
    
    if position_in_line == 1:               # PN in first position
        r = key_post.get(line[-1], r)    # keyword appears in same line in last position
        r = key_post.get(nextline[0], r) # or in the next line in first position
        if len(line) > 1:                   # additional words are attributes of the name
            attr = line[1:]
            lastwords = ' '.join(line[-2:]) # two-word keywords (as in šu ba-ti)
            r = key_post.get(lastwords, r)
        if len(nextline) > 1:               # two-word keyword appearing in firstposition in next line
            firstwords = ' '.join(nextline[0:2])
            r = key_post.get(firstwords, r)
            
    elif position_in_line == 2:             # PN appears in second position with keyword preceding
        r = key_pre.get(line[0], r)        # ŋiri₃ PN or ki PN-ta
        if line[0] == 'ki[place]N' and list(text.loc[text.id_line == lineno, 'form'])[-1].endswith('-še₃'): 
            r = 'destination'              # special case: ki PN-še₃
        if len(line) > 2:
            attr = line[2:]
    else:
        PN = [w for w in line[:position_in_line-1] if w in list(PNs['lemma'])] 
        if PN:
            if line[position_in_line -2] == 'u[and]CNJ':
                r = role[-1]               # same role as previous PN
            else:
                r = 'relation'             # as in PN dumu PN
        else:                              # if there is no preceding PN: drop the name
            r = 'none'                     # this may  need refinement
    if r == '' :
        if treasure[Pno] == 'expenditure': # default role for zi-ga/ba-zi texts
            r = 'recipient'
        elif treasure[Pno] == 'intake' or treasure[Pno] == 'transfer':    # default role for mu-DU texts
            r = 'source'
    role.append(r)
    attribute.append(' '.join(attr))
PNs['role'] = role
PNs['attribute'] = attribute

# Show results in a table with links for checking

In [ ]:
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
PNs2 = PNs.copy()
PNs2['id_word'] = [anchor.format(val,val) for val in PNs['id_word']]

In [ ]:
@interact(rows = (1, len(PNs2), 1))
def showpns(rows = 25): 
    return PNs2.loc[PNs2.id_text == 'P124318', ['id_word', 'form', 'pos', 'lemma', 'role', 'attribute']][:rows].style

# Create Edges
Needs checking. Deal with roles 'offerer' and 'reason'

In [ ]:
intake = [i for i in ids if treasure[i] == 'intake']

In [ ]:
edges = []
for i in intake:                              # one text ID at a time
    people = PNs.index[PNs.id_text == i]    # indexes of all the people in that text
    for p in people:                       # iterate over those indexes
        source = ''
        target = ''
        role = PNs.loc[p]['role']
        if role in ['intermediary', 'representative']:
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p]
            if q:
                for r in q:
                    if PNs.loc[r]['role'] in ['recipient', 'sealer']:
                        target = PNs.loc[r]['lemma']
                        break
        elif role in ['sender', 'deliverer', 'source']:
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p]
            if q:
                for r in q:
                    if PNs.loc[r]['role'] in ['recipient', 'sealer', 'intermediary']:
                        target = PNs.loc[r]['lemma']
                        break
        elif role == 'relation':
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n < p]
            if q:
                target = PNs.loc[q[-1]]['lemma']
        if source and target:
            edges.append([source, target, i])

In [ ]:
edgs = pd.DataFrame(edges)
edgs.columns = ['source', 'target', 'id_text']
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
edgs2 = edgs.copy()
edgs2['id_text'] = [anchor.format(val,val) for val in edgs['id_text']]

In [ ]:
@interact(rows = (1, len(edgs2), 1))
def showedges(rows = 25): 
    return edgs2[:rows].style

In [ ]:
mudus = []
deliverer = ''
for i in intake:
    people = PNs.index[PNs.id_text == i].tolist()
    text = words.loc[words.id_text == i]
    mudu = text.index[text.lemma == 'mu.DU[delivery]N'].tolist()[0]
    p = [p for p in people if p < mudu]
    if len(p) > 0:
        deliverer = [PNs.loc[max(p), 'lemma'], i]
    else:
        deliverer = ['[...]', i]
    mudus.append(deliverer)     

In [ ]:
words.loc[words.id_text == 'P102169']